In [1]:
#import basic system functions
import os
import datetime
from collections import Counter

#import libraries for data wrangling
import pandas as pd
import numpy as np

#import libraries for plotting data
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#import libraries for time series analysis
from fbprophet import Prophet

#import libraries to evaluate model performance
import sklearn.metrics as metrics

In [133]:
#create dataframe with original data that will not be touched throughout notebook
original_data = pd.read_csv('meteolytix_Artikelgruppen_Umsatz_verschiedeneStandorte.csv') 

In [134]:
#set Datum to datetime format
original_data['Datum'] = pd.to_datetime(original_data['Datum'])

In [135]:
#convert date to index
original_data.set_index('Datum', inplace=True)
original_data.sort_index(inplace=True)

# Split dataset into train, validation & test

In [136]:
train_df = original_data[original_data.index.year < 2018]
validation_df = original_data[original_data.index.year == 2018]
test_df = original_data[original_data.index.year == 2019]
corona_df = original_data[original_data.index.year == 2020]

In [137]:
y_true = test_df.loc[(test_df.Filiale == 1)&(test_df.Artikelgruppe == 1)]['Umsatz'].sort_index()

In [138]:
y_true.shift(7)

Datum
2019-01-02           NaN
2019-01-03           NaN
2019-01-04           NaN
2019-01-05           NaN
2019-01-07           NaN
2019-01-08           NaN
2019-01-09           NaN
2019-01-10     65.930828
2019-01-11     60.182502
2019-01-12     45.668989
2019-01-16     46.277807
2019-01-17     88.305986
2019-02-07     46.021931
2019-02-08     24.349444
2019-02-09     56.621904
2019-02-11     36.955666
2019-02-12     65.091269
2019-02-13     50.324744
2019-02-14     38.172480
2019-02-15     88.204022
2019-02-16     49.219815
2019-02-18     48.846050
2019-02-19     46.345937
2019-02-20     56.677268
2019-02-21     39.433361
2019-02-22     76.890155
2019-02-23     50.560181
2019-02-25     46.876766
2019-02-26    105.964834
2019-02-27     70.460470
                 ...    
2019-11-25     69.269946
2019-11-26     45.876670
2019-11-27     35.198984
2019-11-28     47.715045
2019-11-29     50.735178
2019-11-30     64.481440
2019-12-02     59.314603
2019-12-03     53.913301
2019-12-04     70.2

# Build pipeline to transform dataframes to desired format and add features

# Build naive prediction model as baseline

In [139]:
def naive_lag_prediction(y_true, lag=7):
    return y_true.shift(lag)    

In [ ]:
naive_model_performance= pd.DataFrame(columns=['Filiale','Artikelgruppe','MAE','MSE','R2'])

In [165]:
for fil in train_df.Filiale.unique():
    for article in train_df.Artikelgruppe.unique():
        #generate series with true values
        y_true = train_df.loc[(train_df['Filiale'] == fil)&(train_df['Artikelgruppe'] == article)]['Umsatz'].sort_index(ascending=True)
        
        #predict values of series with value from 7 periods before (predict value for next Monday with value from this Monday)
        y_predict = naive_lag_prediction(y_true,lag=7)
         
        #delete first 7 digits for both series, because prediction contains NAN values
        y_true = y_true[7:]
        y_predict = y_predict[7:]
        
        #get model performance metrics
        mae = metrics.mean_absolute_error(y_true,y_predict)
        mse = metrics.mean_squared_error(y_true,y_predict)
        r2 = metrics.r2_score(y_true,y_predict)
        
        naive_model_performance = naive_model_performance.append({'Filiale':fil,'Artikelgruppe':article,'MAE':mae,'MSE':mse,'R2':r2}, ignore_index=True)
        

In [166]:
naive_model_performance

,Filiale,Artikelgruppe,MSE,MAE,R2
0,2.0,1.0,305.055805,13.861837,-0.822229
1,2.0,2.0,357.066192,14.862158,-0.477559
2,2.0,8.0,575.536073,18.761657,0.148034
3,2.0,5.0,389.388460,13.922044,-0.710698
4,2.0,9.0,586.672534,18.915792,-0.417615
5,2.0,11.0,13402.148108,41.055783,-0.725361
6,2.0,4.0,405.437478,15.184859,-0.616785
7,2.0,12.0,1427.137788,27.821560,0.252721
8,2.0,3.0,499.530440,17.419732,-0.764388
9,2.0,7.0,1067.664379,23.295831,0.538868
